In [1]:
print('sss')

sss


In [2]:
import pandas as pd

In [3]:
## read the file
path=r'C:\Users\arvinmvn\Desktop\Ranking\Ranking_IN_Sample_Output.xlsx'
df=pd.read_excel(path)

df.head()

,provider_id,service_name
0,4f796148-38fc-441d-857e-c8db05ca6309,Excess Inventory
1,901aa6b4-267f-483b-b6b1-bb6a606452ea,Storage
2,0ff82c94-f00a-4ce3-ad97-90525a5a2d23,Seller Reinstatement
3,3b1b5637-fe6d-4b3a-bed8-915a6662862d,Accounting
4,ef573518-fc0c-4f61-82d3-6898d0f73484,Advertising Optimization


We will create a view that has provider id, service name,Standard Metrics 1,Standard Metrics 2,Standard Metrics 3,Standard Metrics 4,Standard Metrics 5,Standard Metrics 6,Service Metrics 1,Service Metrics 2

after that we will assign the weights and calculate scores by creating blocks for each service

lets take individual sheets 1st, each sheet has provider id vs service vs metric, we will take it in different data frames and go a join on provider_id and service

In [4]:
provider_list=pd.read_excel(path,sheet_name='provider_list')

In [5]:
ytc_percent_in_week=pd.read_excel(path,sheet_name='YTC % (In week)')
#ytc_percent_in_week=ytc_percent_in_week[['providerid','service','ytc_in_week_percent']]

In [6]:
ytc_sla_ltd_21_7_days=pd.read_excel(path,sheet_name='YTC SLA (LTD 21-7 days)')
ytc_sla_ltd_21_7_days=ytc_sla_ltd_21_7_days[['providerid','service','ytc_sla_21_7']]

In [7]:
ytc_percent_ltd_21_7_days=pd.read_excel(path,sheet_name='YTC % (LTD 21 - 7 days)')
ytc_percent_ltd_21_7_days=ytc_percent_ltd_21_7_days[ytc_percent_ltd_21_7_days['marketplaceid'].notnull()]
ytc_percent_ltd_21_7_days=ytc_percent_ltd_21_7_days[['providerid','service','ytc_ltd_21_7_percent']]

In [8]:
snr_orders_completed=pd.read_excel(path,sheet_name='snr_orders(completed)')
snr_orders_completed['service']='Seller Reinstatement'

In [9]:
cat_orders_completed=pd.read_excel(path,sheet_name='cat_orders(completed)')
cat_orders_completed['service']='Cataloguing'

In [10]:
orders_completed=pd.read_excel(path,sheet_name='Orders(completed)')

## Add SNR orders

## we will replace the services keywords from orders query so that join is mapped correctly, we will also add snr into this

orders_completed=orders_completed.replace( ['Advertise','Boost','FBA','Enhanced Brand Content'], ['Advertising Optimization','Account Management','FBA Preparation','Enhanced Brand Content'] )

orders_completed=pd.concat([orders_completed[['providerid','service','total_4w_orders']],
           snr_orders_completed[snr_orders_completed['metric']=='No. of orders'][['providerid','service','orders']].rename(columns = {'orders': 'total_4w_orders'}, inplace = False)
          ])

## add calalog orders
orders_completed=pd.concat([orders_completed,cat_orders_completed.rename(columns = {'no_orders': 'total_4w_orders'}, inplace = False)])

In [11]:
avg_feedback=pd.read_excel(path,sheet_name='avg_feedback')

In [12]:
unique_seller=pd.read_excel(path,sheet_name='unique_seller')
unique_seller=unique_seller.replace( ['Advertise','Boost','FBA','Enhanced Brand Content'], ['Advertising Optimization','Account Management','FBA Preparation','Enhanced Brand Content'] )
unique_seller=pd.concat([unique_seller,snr_orders_completed[snr_orders_completed['metric']=='No. of sellers'][['providerid','service','orders']].rename(columns = {'orders': 'adoption'}, inplace = False)
          ])        

In [13]:
pip_percent=pd.read_excel(path,sheet_name='pip_%')

pip_percent=pip_percent[['providerid','service','pip_percent']]

In [14]:
caq=pd.read_excel(path,sheet_name='CAQ')
caq['service']='Cataloguing'

In [15]:
cat_asins_created=pd.read_excel(path,sheet_name='cat_asins_created')
cat_asins_created['service']='Cataloguing'

In [16]:
ad_revenue=pd.read_excel(path,sheet_name='Ad Revenue')
ad_revenue['service']='Advertising Optimization'

ad_revenue=ad_revenue.groupby(['providerid','service'])['sp_revenue','attributed_gms','budget','gms','ad_revenue','acos_p'].sum().reset_index()
ad_revenue=ad_revenue[['providerid','ad_revenue','service']]

C:\Users\arvinmvn\AppData\Local\Temp\ipykernel_11976\2856906050.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ad_revenue=ad_revenue.groupby(['providerid','service'])['sp_revenue','attributed_gms','budget','gms','ad_revenue','acos_p'].sum().reset_index()


In [17]:
ad_acos=pd.read_excel(path,sheet_name='ad_ACOS')
ad_acos['service']='Advertising Optimization'
ad_acos=ad_acos.groupby(['providerid','service'])['sp_revenue','attributed_gms','budget','gms','ad_revenue','acos_p'].sum().reset_index()
ad_acos=ad_acos[['providerid','acos_p','service']]

C:\Users\arvinmvn\AppData\Local\Temp\ipykernel_11976\3373560619.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ad_acos=ad_acos.groupby(['providerid','service'])['sp_revenue','attributed_gms','budget','gms','ad_revenue','acos_p'].sum().reset_index()


In [18]:
AM_gms=pd.read_excel(path,sheet_name='AM_gms')
AM_gms['service']='Account Management'
AM_gms=AM_gms.rename(columns = {'gms': 'boost_gms'}, inplace = False)

In [19]:
ebc_gv=pd.read_excel(path,sheet_name='ebc_gv')
ebc_gv['service']='Enhanced Brand Content'
ebc_gv=ebc_gv.groupby(['provider_id','service'])['asins_created','lw_gv_platform','lw_gv_3p','lw_gv_3p_total','lw_gv_platform_total','spn_flag','lw_gv_3p_ytd','lw_gv_platform_ytd','lw_gv_3p_total','lw_gv_platform_total'].sum().reset_index()
ebc_gv=ebc_gv.rename(columns = {'asins_created': 'ebc_gv_asins_created'}, inplace = False)

C:\Users\arvinmvn\AppData\Local\Temp\ipykernel_11976\820171744.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ebc_gv=ebc_gv.groupby(['provider_id','service'])['asins_created','lw_gv_platform','lw_gv_3p','lw_gv_3p_total','lw_gv_platform_total','spn_flag','lw_gv_3p_ytd','lw_gv_platform_ytd','lw_gv_3p_total','lw_gv_platform_total'].sum().reset_index()


In [20]:
ebc_asin_coverage=pd.read_excel(path,sheet_name='ebc_asin_coverage')
ebc_asin_coverage['service']='Enhanced Brand Content'
ebc_asin_coverage=ebc_asin_coverage.groupby(['provider_id','service'])['asins_created'].sum().reset_index()
ebc_asin_coverage=ebc_asin_coverage.rename(columns = {'asins_created': 'ebc_asin_coverage_asins_created'}, inplace = False)

In [21]:
tax_unblocked_ttm=pd.read_excel(path,sheet_name='tax_unblocked_ttm')
tax_unblocked_ttm['service']='Taxes'
tax_unblocked_ttm=tax_unblocked_ttm.rename(columns = {'unblocked_gms': 'tax_unblocked_gms'}, inplace = False)

In [22]:
SNR_reinstatement_p_num=pd.read_excel(path,sheet_name='SNR_reinstatement_p_num')

In [23]:
SNR_reinstatement_p_deno=pd.read_excel(path,sheet_name='SNR_reinstatement_p_deno')

In [24]:

## filter out numerator and denomenator from the respective blocks and do a join on provider id vs num vs deno
snr_reinstatement_p=pd.merge(
    SNR_reinstatement_p_num[['providerid','no_sellers']].rename(columns = {'no_sellers': 'snr_reinst_p_num'}, inplace = False),
    SNR_reinstatement_p_deno[['providerid','ytd']].rename(columns = {'ytd': 'snr_reinst_p_deno'}, inplace = False),
    how='outer', 
    left_on =['providerid'],
    right_on=['providerid']
    )
## create a view on providerid vs (num/deno) and fill na with o
snr_reinstatement_p['snr_reinst_p']=snr_reinstatement_p['snr_reinst_p_num']/snr_reinstatement_p['snr_reinst_p_deno']
snr_reinstatement_p=snr_reinstatement_p.fillna(0).round(4)
snr_reinstatement_p['service']='Seller Reinstatement'
snr_reinstatement_p=snr_reinstatement_p[['providerid','snr_reinst_p','service']]

In [25]:
ds_ncns=pd.read_excel(path,sheet_name='DS_NCNS')
ds_ncns['service']='Domestic Shipping'

In [26]:
compliance_unblocked_gms=pd.read_excel(path,sheet_name='compliance_unblocked_gms')
compliance_unblocked_gms['service']='Compliance'
compliance_unblocked_gms=compliance_unblocked_gms.rename(columns = {'unblocked_gms': 'compliance_unblocked_gms'}, inplace = False)

In [27]:
join_1_2=pd.merge(
    provider_list,
    ytc_percent_in_week.rename(columns = {'providerid': 'providerid_1', 'service': 'service_1'}, inplace = False),
    how='left',
    left_on=['provider_id','service_name'],
    right_on=['providerid_1','service_1']
    )

In [28]:
len(join_1_2)

5975

In [29]:
join_1_2.columns

Index(['provider_id', 'service_name', 'providerid_1', 'service_1',
       'ytc_in_week_numerator', 'ytc_in_week_denomenator',
       'ytc_in_week_percent'],
      dtype='object')

In [30]:
#ytc_sla_ltd_21_7_days.rename(columns = {'providerid': 'providerid_1', 'service': 'service_1'}, inplace = False)

In [31]:
join_1_3=pd.merge(
    join_1_2.drop(['providerid_1','service_1'],axis=1, inplace=False),
    ytc_sla_ltd_21_7_days.rename(columns = {'providerid': 'providerid_2', 'service': 'service_2'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_2','service_2']
    )

In [32]:
len(join_1_3)

5975

In [33]:
join_1_3.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'providerid_2',
       'service_2', 'ytc_sla_21_7'],
      dtype='object')

In [34]:
join_1_4=pd.merge(
    join_1_3.drop(['providerid_2','service_2'],axis=1, inplace=False),
    ytc_percent_ltd_21_7_days.rename(columns = {'providerid': 'providerid_3', 'service': 'service_3'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_3','service_3']
    )

In [35]:
join_1_4.to_clipboard()

In [36]:
len(join_1_4)

5975

In [37]:
join_1_5=pd.merge(
    join_1_4.drop(['providerid_3','service_3'],axis=1, inplace=False),
    orders_completed.rename(columns = {'providerid': 'providerid_4', 'service': 'service_4'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_4','service_4']
    )

In [38]:
len(join_1_5)

5975

In [39]:
join_1_6=pd.merge(
    join_1_5.drop(['providerid_4','service_4'],axis=1, inplace=False),
    cat_orders_completed.rename(columns = {'providerid': 'providerid_5', 'service': 'service_5'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_5','service_5']
    )

In [40]:
len(join_1_6)

5975

In [41]:
snr_reinstatement_p.columns

Index(['providerid', 'snr_reinst_p', 'service'], dtype='object')

In [42]:
join_1_7=pd.merge(
    join_1_6.drop(['providerid_5','service_5'],axis=1, inplace=False),
    snr_reinstatement_p.rename(columns = {'providerid': 'providerid_6', 'service': 'service_6'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_6','service_6']
    )

In [43]:
len(join_1_7)

5975

In [44]:
join_1_8=pd.merge(
    join_1_7.drop(['providerid_6','service_6'],axis=1, inplace=False),
    avg_feedback.rename(columns = {'provider_id': 'providerid_7', 'services_applicable': 'service_7'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_7','service_7']
    )

In [45]:
len(join_1_8)

5975

In [46]:
join_1_9=pd.merge(
    join_1_8.drop(['providerid_7','service_7'],axis=1, inplace=False),
    unique_seller.rename(columns = {'providerid': 'providerid_8', 'service': 'service_8'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_8','service_8']
    )

In [47]:
len(join_1_9)

5975

In [48]:
join_1_10=pd.merge(
    join_1_9.drop(['providerid_8','service_8'],axis=1, inplace=False),
    pip_percent.rename(columns = {'providerid': 'providerid_9', 'service': 'service_9'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_9','service_9']
    )

In [49]:
len(join_1_10)

5975

In [50]:
join_1_11=pd.merge(
    join_1_10.drop(['providerid_9','service_9'],axis=1, inplace=False),
    caq.rename(columns = {'provider_id': 'providerid_10', 'service': 'service_10'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_10','service_10']
    )

In [51]:
len(join_1_11)

5975

In [52]:
join_1_12=pd.merge(
    join_1_11.drop(['providerid_10','service_10'],axis=1, inplace=False),
    cat_asins_created.rename(columns = {'provider_name': 'providerid_11', 'service': 'service_11'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_11','service_11']
    )

In [53]:
len(join_1_12)

5975

In [54]:
join_1_13=pd.merge(
    join_1_12.drop(['providerid_11','service_11'],axis=1, inplace=False),
    ad_revenue.rename(columns = {'providerid': 'providerid_12', 'service': 'service_12'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_12','service_12']
    )

In [55]:
join_1_14=pd.merge(
    join_1_13.drop(['providerid_12','service_12'],axis=1, inplace=False),
    ad_acos.rename(columns = {'providerid': 'providerid_13', 'service': 'service_13'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_13','service_13']
    )

In [56]:
len(join_1_14)

5975

In [57]:
join_1_15=pd.merge(
    join_1_14.drop(['providerid_13','service_13'],axis=1, inplace=False),
    AM_gms.rename(columns = {'providerid': 'providerid_14', 'service': 'service_14'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_14','service_14']
    )

In [58]:
len(join_1_15)

5975

In [59]:
join_1_16=pd.merge(
    join_1_15.drop(['providerid_14','service_14'],axis=1, inplace=False),
    ebc_gv.rename(columns = {'provider_id': 'providerid_15', 'service': 'service_15'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_15','service_15']
    )

In [60]:
len(join_1_16)

5975

In [61]:
join_1_17=pd.merge(
    join_1_16.drop(['providerid_15','service_15'],axis=1, inplace=False),
    ebc_asin_coverage.rename(columns = {'provider_id': 'providerid_16', 'service': 'service_16'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_16','service_16']
    )

In [62]:
len(join_1_17)

5975

In [63]:
join_1_18=pd.merge(
    join_1_17.drop(['providerid_16','service_16'],axis=1, inplace=False),
    tax_unblocked_ttm.rename(columns = {'providerid': 'providerid_17', 'service': 'service_17'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_17','service_17']
    )

In [64]:
len(join_1_18)

5975

In [65]:
join_1_19=pd.merge(
    join_1_18.drop(['providerid_17','service_17'],axis=1, inplace=False),
    ds_ncns.rename(columns = {'provider_id': 'providerid_18', 'service': 'service_18'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_18','service_18']
    )

In [66]:
len(join_1_19)

5975

In [67]:
join_1_20=pd.merge(
    join_1_19.drop(['providerid_18','service_18'],axis=1, inplace=False),
    compliance_unblocked_gms.rename(columns = {'providerid': 'providerid_19', 'service': 'service_19'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_19','service_19']
    )

In [68]:
len(join_1_20)

5975

In [69]:
join_1_20.to_clipboard()

In [70]:
join_1_20.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'ytc_sla_21_7',
       'ytc_ltd_21_7_percent', 'total_4w_orders', 'no_orders', 'snr_reinst_p',
       'avg_feedback', 'adoption', 'pip_percent', 'caq_score',
       'count_of_asins', 'ad_revenue', 'acos_p', 'boost_gms',
       'ebc_gv_asins_created', 'lw_gv_platform', 'lw_gv_3p', 'lw_gv_3p_total',
       'lw_gv_platform_total', 'spn_flag', 'lw_gv_3p_ytd',
       'lw_gv_platform_ytd', 'lw_gv_3p_total', 'lw_gv_platform_total',
       'ebc_asin_coverage_asins_created', 'tax_unblocked_gms', 'ncns',
       'providerid_19', 'unblocked_asin', 'compliance_unblocked_gms',
       'service_19'],
      dtype='object')

In [71]:
## we have the metrics view prepared, few niggles that need to be checked are
# provider id list for cataloging for those which has that spn codes

In [72]:
## Ranking view for imaging

In [73]:
## let's have the weights for imaging in this block

ytc_percent_in_week_percent_imaging_wt= 10/100
ytc_sla_ltd_21_7_days_imaging_wt= 5/100
ytc_percent_ltd_21_7_days_imaging_wt= 5/100
pip_percent_imaging_wt= 10/100
orders_completed_imaging_wt= 35/100
unique_seller_imaging_wt= 15/100
avg_feedback_imaging_wt= 20/100


In [74]:
join_1_20.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'ytc_sla_21_7',
       'ytc_ltd_21_7_percent', 'total_4w_orders', 'no_orders', 'snr_reinst_p',
       'avg_feedback', 'adoption', 'pip_percent', 'caq_score',
       'count_of_asins', 'ad_revenue', 'acos_p', 'boost_gms',
       'ebc_gv_asins_created', 'lw_gv_platform', 'lw_gv_3p', 'lw_gv_3p_total',
       'lw_gv_platform_total', 'spn_flag', 'lw_gv_3p_ytd',
       'lw_gv_platform_ytd', 'lw_gv_3p_total', 'lw_gv_platform_total',
       'ebc_asin_coverage_asins_created', 'tax_unblocked_gms', 'ncns',
       'providerid_19', 'unblocked_asin', 'compliance_unblocked_gms',
       'service_19'],
      dtype='object')

In [75]:
## imaging scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

imag_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

imag_rank=imag_rank[imag_rank['service_name']=='Imaging']
imag_rank=imag_rank.fillna(0)

##prepare the score

imag_rank['score']=(  ytc_percent_in_week_percent_imaging_wt*imag_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_imaging_wt*imag_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_imaging_wt*imag_rank['ytc_ltd_21_7_percent']
                    + pip_percent_imaging_wt*imag_rank['pip_percent']
                    + orders_completed_imaging_wt*imag_rank['total_4w_orders']
                    + unique_seller_imaging_wt*imag_rank['adoption']
                    + avg_feedback_imaging_wt*imag_rank['avg_feedback']
                   )

imag_rank['rank']=imag_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
imag_rank = imag_rank.sort_values(by="rank", ignore_index=True)
imag_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Imaging,0.00,0.20,0.0,40.74,2405.0,95.0,5.00,861.084,1.0
1,a5c32bce-5fe4-4f64-8764-7987f12eab79,Imaging,0.00,0.00,0.0,60.47,1073.0,0.0,5.00,382.597,2.0
2,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Imaging,0.00,0.00,0.0,60.00,651.0,1.0,5.00,235.000,3.0
3,de030dd5-7b08-4987-8f4d-554aaf98f8f7,Imaging,0.00,0.00,0.0,60.00,465.0,0.0,4.33,169.616,4.0
4,348d6610-bf8c-40e0-985e-4c913b7d1f78,Imaging,20.59,12.12,0.0,40.00,189.0,0.0,5.00,73.815,5.0


In [76]:
## let's have the weights for Accounting in this block

ytc_percent_in_week_percent_accounting_wt= 20/100
ytc_sla_ltd_21_7_days_accounting_wt= 5/100
ytc_percent_ltd_21_7_days_accounting_wt= 5/100
#pip_percent_accounting_wt= 10/100
orders_completed_accounting_wt= 30/100
unique_seller_accounting_wt= 20/100
avg_feedback_accounting_wt= 20/100

In [77]:
## Accounting scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

accounting_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

accounting_rank=accounting_rank[accounting_rank['service_name']=='Accounting']
accounting_rank=accounting_rank.fillna(0)

##prepare the score

accounting_rank['score']=(  ytc_percent_in_week_percent_accounting_wt*accounting_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_accounting_wt*accounting_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_accounting_wt*accounting_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_accounting_wt*accounting_rank['pip_percent']
                    + orders_completed_accounting_wt*accounting_rank['total_4w_orders']
                    + unique_seller_accounting_wt*accounting_rank['adoption']
                    + avg_feedback_accounting_wt*accounting_rank['avg_feedback']
                   )

accounting_rank['rank']=accounting_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
accounting_rank = accounting_rank.sort_values(by="rank", ignore_index=True)
accounting_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,c9f3c2fb-0628-485b-8308-4a03c8699434,Accounting,0.0,0.0,0.0,59.26,127.0,73.0,5.0,53.7,1.0
1,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Accounting,0.0,0.0,0.0,75.00,83.0,82.0,5.0,42.3,2.0
2,289fbe89-60de-4d0c-b279-28231502e9ba,Accounting,0.0,0.0,0.0,0.00,25.0,22.0,5.0,12.9,3.0
3,6af946b2-d8b9-4499-8005-90024d62404d,Accounting,0.0,0.0,0.0,0.00,20.0,0.0,0.0,6.0,4.0
4,04407379-d775-4a69-9d63-bf5b5e01b282,Accounting,0.0,0.0,0.0,100.00,10.0,6.0,0.0,4.2,5.0


In [78]:
## let's have the weights for Training in this block

ytc_percent_in_week_percent_training_wt= 20/100
ytc_sla_ltd_21_7_days_training_wt= 5/100
ytc_percent_ltd_21_7_days_training_wt= 5/100
#pip_percent_training_wt= 10/100
orders_completed_training_wt= 30/100
unique_seller_training_wt= 20/100
avg_feedback_training_wt= 20/100

In [79]:
##Training scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

training_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

training_rank=training_rank[training_rank['service_name']=='Training']
training_rank=training_rank.fillna(0)

##prepare the score

training_rank['score']=(  ytc_percent_in_week_percent_training_wt*training_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_training_wt*training_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_training_wt*training_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_training_wt*training_rank['pip_percent']
                    + orders_completed_training_wt*training_rank['total_4w_orders']
                    + unique_seller_training_wt*training_rank['adoption']
                    + avg_feedback_training_wt*training_rank['avg_feedback']
                   )

training_rank['rank']=training_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
training_rank = training_rank.sort_values(by="rank", ignore_index=True)
training_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,26ddd595-c3a2-413f-bec2-2de54264ede8,Training,0.0,5.49,0.0,55.56,399.0,0.0,1.00,120.1745,1.0
1,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Training,0.0,0.00,0.0,80.00,178.0,0.0,0.00,53.4000,2.0
2,b8e4a5b0-a3df-4d60-8394-f1dc81bcad6c,Training,0.0,0.00,0.0,0.00,175.0,0.0,5.00,53.5000,3.0
3,cd11dd3a-1103-46b8-a25e-ea02580181f3,Training,0.0,0.00,0.0,87.50,131.0,0.0,5.00,40.3000,4.0
4,0611cfd2-2c7f-44e1-9a2c-e6f60cdca5cd,Training,0.0,0.00,0.0,0.00,127.0,1.0,4.16,39.1320,5.0


In [80]:
## let's have the weights for International Shipping in this block

ytc_percent_in_week_percent_i_s_wt= 20/100
ytc_sla_ltd_21_7_days_i_s_wt= 5/100
ytc_percent_ltd_21_7_days_i_s_wt= 5/100
#pip_percent_i_s_wt= 10/100
orders_completed_i_s_wt= 30/100
unique_seller_i_s_wt= 20/100
avg_feedback_i_s_wt= 20/100

In [81]:
##International Shipping scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

i_s_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

i_s_rank=i_s_rank[i_s_rank['service_name']=='International Shipping']
i_s_rank=i_s_rank.fillna(0)

##prepare the score

i_s_rank['score']=(  ytc_percent_in_week_percent_i_s_wt*i_s_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_i_s_wt*i_s_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_i_s_wt*i_s_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_i_s_wt*i_s_rank['pip_percent']
                    + orders_completed_i_s_wt*i_s_rank['total_4w_orders']
                    + unique_seller_i_s_wt*i_s_rank['adoption']
                    + avg_feedback_i_s_wt*i_s_rank['avg_feedback']
                   )

i_s_rank['rank']=i_s_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
i_s_rank = i_s_rank.sort_values(by="rank", ignore_index=True)
i_s_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,3297ad3e-a99f-46b1-ab89-e0ef9960b03d,International Shipping,100.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,1.0
1,6a9c04cb-4c34-4fa6-8d4a-b65561aeec1c,International Shipping,100.0,0.0,100.0,0.0,0.0,0.0,0.0,25.0,1.0
2,9b190b53-784f-4e15-a904-fe5f5707d2c6,International Shipping,100.0,25.0,75.0,0.0,0.0,0.0,0.0,25.0,1.0
3,c35b81cb-c6b6-4f03-8c9d-bbcbe49dc027,International Shipping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,2eb5a4bd-7a3c-49a1-a5be-be674e37784f,International Shipping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [82]:
## let's have the weights for Translation in this block

ytc_percent_in_week_percent_trans_wt= 20/100
ytc_sla_ltd_21_7_days_trans_wt= 5/100
ytc_percent_ltd_21_7_days_trans_wt= 5/100
#pip_percent_trans_wt= 10/100
orders_completed_trans_wt= 30/100
unique_seller_trans_wt= 20/100
avg_feedback_trans_wt= 20/100

In [83]:
##Translation scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

trans_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

trans_rank=trans_rank[trans_rank['service_name']=='Translation']
trans_rank=trans_rank.fillna(0)

##prepare the score

trans_rank['score']=(  ytc_percent_in_week_percent_trans_wt*trans_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_trans_wt*trans_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_trans_wt*trans_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_i_s_wt*trans_rank['pip_percent']
                    + orders_completed_trans_wt*trans_rank['total_4w_orders']
                    + unique_seller_trans_wt*trans_rank['adoption']
                    + avg_feedback_trans_wt*trans_rank['avg_feedback']
                   )

trans_rank['rank']=trans_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
trans_rank = trans_rank.sort_values(by="rank", ignore_index=True)
trans_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,db84ca33-2ba1-4bf9-ba4c-be90ce9746bb,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,17b3aaab-2ebc-4a99-b6d1-642d36f7b0e4,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,acf33907-2e82-41af-b23b-abb74976ec47,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,5cbda68b-9b12-4b83-a044-a10a92baa99c,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,e96e6f84-8d07-4d48-89fe-40a36bb3f0d2,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [84]:
## let's have the weights for International Returns in this block

ytc_percent_in_week_percent_i_r_wt= 20/100
ytc_sla_ltd_21_7_days_i_r_wt= 5/100
ytc_percent_ltd_21_7_days_i_r_wt= 5/100
#pip_percent_i_r_wt= 10/100
orders_completed_i_r_wt= 30/100
unique_seller_i_r_wt= 20/100
avg_feedback_i_r_wt= 20/100

In [85]:
##International Shipping scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

i_r_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

i_r_rank=i_r_rank[i_r_rank['service_name']=='International Returns']
i_r_rank=i_r_rank.fillna(0)

##prepare the score

i_r_rank['score']=(  ytc_percent_in_week_percent_i_r_wt*i_r_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_i_r_wt*i_r_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_i_r_wt*i_r_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_i_r_wt*i_r_rank['pip_percent']
                    + orders_completed_i_r_wt*i_r_rank['total_4w_orders']
                    + unique_seller_i_r_wt*i_r_rank['adoption']
                    + avg_feedback_i_r_wt*i_r_rank['avg_feedback']
                   )

i_r_rank['rank']=i_r_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
i_r_rank = i_r_rank.sort_values(by="rank", ignore_index=True)
i_r_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,ce11ae06-6774-49d5-abc4-70a805b7826e,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,9538a048-7e19-49da-88e6-f1ebccd46069,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,b33f6d0b-2fd6-4fc5-afbe-96865e5e5cf7,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,e07d88c6-9ecb-4991-a27f-aa3f04ca2814,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,71291d76-43f5-4b91-bdcf-713080294d85,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [86]:
## let's have the weights for FBA in this block

ytc_percent_in_week_percent_fba_wt= 20/100
ytc_sla_ltd_21_7_days_fba_wt= 5/100
ytc_percent_ltd_21_7_days_fba_wt= 5/100
#pip_percent_fba_wt= 10/100
orders_completed_fba_wt= 30/100
unique_seller_fba_wt= 20/100
avg_feedback_fba_wt= 20/100

In [87]:
## FBA scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

fba_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

fba_rank=fba_rank[fba_rank['service_name']=='FBA Preparation']
fba_rank=fba_rank.fillna(0)

##prepare the score

fba_rank['score']=(  ytc_percent_in_week_percent_fba_wt*fba_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_fba_wt*fba_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_fba_wt*fba_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_fba_wt*fba_rank['pip_percent']
                    + orders_completed_fba_wt*fba_rank['total_4w_orders']
                    + unique_seller_fba_wt*fba_rank['adoption']
                    + avg_feedback_fba_wt*fba_rank['avg_feedback']
                   )

fba_rank['rank']=fba_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
fba_rank = fba_rank.sort_values(by="rank", ignore_index=True)
fba_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,b8e4a5b0-a3df-4d60-8394-f1dc81bcad6c,FBA Preparation,0.0,0.0,0.0,0.00,137.0,1.0,4.28,42.156,1.0
1,504878ee-ff2f-46b3-98f7-77d5e7ed139b,FBA Preparation,0.0,0.0,0.0,22.22,44.0,40.0,5.00,22.200,2.0
2,d2f317ff-cbd0-4846-9b47-18fe8273c6ae,FBA Preparation,0.0,0.0,0.0,16.67,17.0,11.0,0.00,7.300,3.0
3,9024e1be-44b7-4d2e-9d2d-6db10e8c0ded,FBA Preparation,0.0,0.0,0.0,0.00,12.0,7.0,5.00,6.000,4.0
4,a021786e-3c21-48be-955a-549382f3a3ae,FBA Preparation,0.0,0.0,0.0,0.00,8.0,8.0,5.00,5.000,5.0


In [88]:
## let's have the weights for storage in this block

ytc_percent_in_week_percent_stor_wt= 20/100
ytc_sla_ltd_21_7_days_stor_wt= 5/100
ytc_percent_ltd_21_7_days_stor_wt= 5/100
#pip_percent_stor_wt= 10/100
orders_completed_stor_wt= 30/100
unique_seller_stor_wt= 20/100
avg_feedback_stor_wt= 20/100

In [89]:
## Storage scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

stor_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

stor_rank=stor_rank[stor_rank['service_name']=='Storage']
stor_rank=stor_rank.fillna(0)

##prepare the score

stor_rank['score']=(  ytc_percent_in_week_percent_stor_wt*stor_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_stor_wt*stor_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_stor_wt*stor_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_stor_wt*stor_rank['pip_percent']
                    + orders_completed_stor_wt*stor_rank['total_4w_orders']
                    + unique_seller_stor_wt*stor_rank['adoption']
                    + avg_feedback_stor_wt*stor_rank['avg_feedback']
                   )

stor_rank['rank']=stor_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
stor_rank = stor_rank.sort_values(by="rank", ignore_index=True)
stor_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,fb45d4ec-2467-4035-a153-93ec5331d753,Storage,16.67,1.65,6.61,0.0,144.0,133.0,5.0,74.5470,1.0
1,ad3442f1-f0c5-4a84-b3c3-64a61fe4dbe7,Storage,6.25,36.84,0.00,0.0,64.0,46.0,5.0,32.4920,2.0
2,bbef9f87-0261-4212-9b6e-60922e19c662,Storage,28.57,66.67,0.00,0.0,44.0,50.0,0.0,32.2475,3.0
3,bff421b8-1ade-471d-a184-b2c45d69113d,Storage,0.00,0.00,0.00,0.0,40.0,38.0,5.0,20.6000,4.0
4,0901c78c-5389-4186-9a87-8aebdc23b710,Storage,0.00,0.00,0.00,0.0,31.0,31.0,0.0,15.5000,5.0


In [90]:
## let's have the weights for IPA in this block

ytc_percent_in_week_percent_ipa_wt= 20/100
ytc_sla_ltd_21_7_days_ipa_wt= 5/100
ytc_percent_ltd_21_7_days_ipa_wt= 5/100
#pip_percent_ipa_wt= 10/100
orders_completed_ipa_wt= 30/100
unique_seller_ipa_wt= 20/100
avg_feedback_ipa_wt= 20/100

In [91]:
## IPA scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

ipa_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

ipa_rank=ipa_rank[ipa_rank['service_name']=='IP Accelerator']
ipa_rank=ipa_rank.fillna(0)

##prepare the score

ipa_rank['score']=(  ytc_percent_in_week_percent_ipa_wt*ipa_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_ipa_wt*ipa_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_ipa_wt*ipa_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_ipa_wt*ipa_rank['pip_percent']
                    + orders_completed_ipa_wt*ipa_rank['total_4w_orders']
                    + unique_seller_ipa_wt*ipa_rank['adoption']
                    + avg_feedback_ipa_wt*ipa_rank['avg_feedback']
                   )

ipa_rank['rank']=ipa_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
ipa_rank = ipa_rank.sort_values(by="rank", ignore_index=True)
ipa_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,fe763b02-f44d-4129-810a-3020d7afde04,IP Accelerator,0.0,0.0,0.0,2.78,0.0,0.0,5.0,1.0,1.0
1,70729788-8930-43c2-aff6-3f33fdc921d3,IP Accelerator,0.0,0.0,0.0,54.84,0.0,0.0,5.0,1.0,1.0
2,41f3f250-fa8b-4bf5-b39e-18a3eb41cd76,IP Accelerator,100.0,0.0,100.0,0.00,0.0,0.0,0.0,25.0,2.0
3,c50388ea-f913-4cf1-aa47-7d9fb8ca8aaf,IP Accelerator,50.0,100.0,0.0,25.00,0.0,0.0,0.0,15.0,3.0
4,d3ba7600-8516-4ed1-b996-d646afa42fd1,IP Accelerator,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,4.0


In [92]:
## let's have the weights for Excess Inventory  in this block

ytc_percent_in_week_percent_exec_wt= 20/100
ytc_sla_ltd_21_7_days_exec_wt= 5/100
ytc_percent_ltd_21_7_days_exec_wt= 5/100
#pip_percent_exec_wt= 10/100
orders_completed_exec_wt= 30/100
unique_seller_exec_wt= 20/100
avg_feedback_exec_wt= 20/100

In [93]:
## Excesss Inventory scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback']

exec_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

exec_rank=exec_rank[exec_rank['service_name']=='Excess Inventory']
exec_rank=exec_rank.fillna(0)

##prepare the score

exec_rank['score']=(  ytc_percent_in_week_percent_exec_wt*exec_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_exec_wt*exec_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_exec_wt*exec_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_exec_wt*exec_rank['pip_percent']
                    + orders_completed_exec_wt*exec_rank['total_4w_orders']
                    + unique_seller_exec_wt*exec_rank['adoption']
                    + avg_feedback_exec_wt*exec_rank['avg_feedback']
                   )

exec_rank['rank']=exec_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
exec_rank = exec_rank.sort_values(by="rank", ignore_index=True)
exec_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,score,rank
0,0a8e300b-68a1-42b5-9ad1-c953710dde00,Excess Inventory,100.0,0.0,100.0,0.0,0.0,0.0,0.0,25.0,1.0
1,5a3bdad8-0f7b-45be-bade-d2e8b13d468f,Excess Inventory,100.0,0.0,100.0,0.0,0.0,0.0,0.0,25.0,1.0
2,b6209911-111c-448f-b152-4d1ccd8a91b9,Excess Inventory,100.0,0.0,100.0,0.0,0.0,0.0,0.0,25.0,1.0
3,bc3c2364-16f3-43b9-b775-17a921b27459,Excess Inventory,100.0,0.0,100.0,0.0,0.0,0.0,0.0,25.0,1.0
4,4f796148-38fc-441d-857e-c8db05ca6309,Excess Inventory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [94]:
## let's have the weights for Cataloging in this block

ytc_percent_in_week_percent_cat_wt= 10/100
ytc_sla_ltd_21_7_days_cat_wt= 10/100
ytc_percent_ltd_21_7_days_cat_wt= 10/100
#pip_percent_cat_wt= 10/100
orders_completed_cat_wt= 20/100
unique_seller_cat_wt= 10/100
avg_feedback_cat_wt= 10/100
caq_cat_wt=20/100
cat_asin_created_wt=10/100

In [95]:
## Excesss Catalouging scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'caq_score',
       'count_of_asins']

cat_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

cat_rank=cat_rank[cat_rank['service_name']=='Cataloguing']
cat_rank=cat_rank.fillna(0)

##prepare the score

cat_rank['score']=(  ytc_percent_in_week_percent_cat_wt*cat_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_cat_wt*cat_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_cat_wt*cat_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_cat_wt*cat_rank['pip_percent']
                    + orders_completed_cat_wt*cat_rank['total_4w_orders']
                    + unique_seller_cat_wt*cat_rank['adoption']
                    + avg_feedback_cat_wt*cat_rank['avg_feedback']
                    + caq_cat_wt*cat_rank['caq_score']
                    + cat_asin_created_wt*cat_rank['count_of_asins']
                   )

cat_rank['rank']=cat_rank[['total_4w_orders','caq_score','avg_feedback','adoption','ytc_in_week_percent','count_of_asins']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
cat_rank = cat_rank.sort_values(by="rank", ignore_index=True)
cat_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,caq_score,count_of_asins,score,rank
0,26ddd595-c3a2-413f-bec2-2de54264ede8,Cataloguing,0.0,5.05,0.0,25.27,821.0,0.0,3.75,0,27830.0,2948.080,1.0
1,b3f2f78a-5443-48ea-8b17-28b337100ccc,Cataloguing,0.0,43.64,0.0,52.63,672.0,0.0,5.00,0,1602143.0,160353.564,2.0
2,860468c6-9e94-4627-8e89-23e58e944fff,Cataloguing,0.0,0.00,0.0,68.40,430.0,0.0,5.00,0,167656.0,16852.100,3.0
3,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Cataloguing,0.0,0.05,0.0,43.58,394.0,0.0,4.93,0,313492.0,31428.498,4.0
4,04407379-d775-4a69-9d63-bf5b5e01b282,Cataloguing,0.0,0.00,0.0,96.47,369.0,0.0,5.00,0,25625.0,2636.800,5.0


In [96]:
## let's have the weights for taxes in this block

ytc_percent_in_week_percent_tax_wt= 20/100
ytc_sla_ltd_21_7_days_tax_wt= 5/100
ytc_percent_ltd_21_7_days_tax_wt= 5/100
#pip_percent_tax_wt= 10/100
orders_completed_tax_wt= 30/100
unique_seller_tax_wt= 20/100
avg_feedback_tax_wt= 20/100

In [97]:
## Excesss Inventory scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'tax_unblocked_gms']

tax_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

tax_rank=tax_rank[tax_rank['service_name']=='Taxes']
tax_rank=tax_rank.fillna(0)

##prepare the score

tax_rank['score']=(  ytc_percent_in_week_percent_tax_wt*tax_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_tax_wt*tax_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_tax_wt*tax_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_tax_wt*tax_rank['pip_percent']
                    + orders_completed_tax_wt*tax_rank['total_4w_orders']
                    + unique_seller_tax_wt*tax_rank['adoption']
                    + avg_feedback_tax_wt*tax_rank['avg_feedback']
                   )

tax_rank['rank']=tax_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
tax_rank = tax_rank.sort_values(by="rank", ignore_index=True)
tax_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,tax_unblocked_gms,score,rank
0,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Taxes,0.0,0.00,0.0,58.33,817.0,524.0,4.9,0.00,350.8800,1.0
1,60f8df89-fa86-4b6d-a48b-140580551e3e,Taxes,0.0,0.61,0.0,55.56,438.0,399.0,0.0,59624071.04,211.2305,2.0
2,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Taxes,0.0,0.00,0.0,100.00,259.0,259.0,5.0,12264070.72,130.5000,3.0
3,8ab05dea-5450-42ed-b06a-b3e49a8e1f05,Taxes,0.0,0.00,0.0,0.00,137.0,101.0,0.0,-790.18,61.3000,4.0
4,256c5d5f-c895-4fd2-aac0-2137738f0c93,Taxes,0.0,0.00,0.0,66.67,115.0,115.0,0.0,6387073.58,57.5000,5.0


In [98]:
## let's have the weights for compliance in this block

ytc_percent_in_week_percent_comp_wt= 20/100
ytc_sla_ltd_21_7_days_comp_wt= 5/100
ytc_percent_ltd_21_7_days_comp_wt= 5/100
#pip_percent_comp_wt= 10/100
orders_completed_comp_wt= 30/100
unique_seller_comp_wt= 20/100
avg_feedback_comp_wt= 20/100

In [99]:
## Excesss Compliance scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'compliance_unblocked_gms',
        'unblocked_asin']

comp_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

comp_rank=comp_rank[comp_rank['service_name']=='Compliance']
comp_rank=comp_rank.fillna(0)

##prepare the score

comp_rank['score']=(  ytc_percent_in_week_percent_comp_wt*comp_rank['ytc_in_week_percent']
                    + ytc_sla_ltd_21_7_days_comp_wt*comp_rank['ytc_sla_21_7']
                    + ytc_percent_ltd_21_7_days_comp_wt*comp_rank['ytc_ltd_21_7_percent']
                    #+ pip_percent_comp_wt*comp_rank['pip_percent']
                    + orders_completed_comp_wt*comp_rank['total_4w_orders']
                    + unique_seller_comp_wt*comp_rank['adoption']
                    + avg_feedback_comp_wt*comp_rank['avg_feedback']
                   )

comp_rank['rank']=comp_rank[['total_4w_orders','avg_feedback','adoption','ytc_in_week_percent']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
comp_rank = comp_rank.sort_values(by="rank", ignore_index=True)
comp_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,compliance_unblocked_gms,unblocked_asin,score,rank
0,8ab05dea-5450-42ed-b06a-b3e49a8e1f05,Compliance,0.0,0.0,0.0,0.00,137.0,101.0,0.0,0,0,61.3,1.0
1,197b7a61-3bea-4990-b900-f5dee72fe481,Compliance,0.0,0.0,0.0,0.00,98.0,0.0,0.0,0,0,29.4,2.0
2,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Compliance,0.0,0.0,0.0,38.46,88.0,88.0,0.0,0,0,44.0,3.0
3,218f9c74-2d4e-4a27-86a2-75faa955d71e,Compliance,0.0,0.0,0.0,40.00,60.0,54.0,0.0,0,0,28.8,4.0
4,60f8df89-fa86-4b6d-a48b-140580551e3e,Compliance,0.0,0.0,0.0,0.00,4.0,4.0,0.0,0,0,2.0,5.0


In [100]:
## for adz account management ebc and DS, we have ytc my understanding is it is ytc to contact numerator.this we have to check 

In [101]:
## let's have the weights for adz in this block

ytc_percent_in_week_num_adz_wt= 20/100
orders_completed_adz_wt= 20/100
unique_seller_adz_wt= 15/100
avg_feedback_adz_wt= 15/100
acos_adz_wt=10/100
ad_revenue_adz_wt=25/100

In [102]:
join_1_20.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'ytc_sla_21_7',
       'ytc_ltd_21_7_percent', 'total_4w_orders', 'no_orders', 'snr_reinst_p',
       'avg_feedback', 'adoption', 'pip_percent', 'caq_score',
       'count_of_asins', 'ad_revenue', 'acos_p', 'boost_gms',
       'ebc_gv_asins_created', 'lw_gv_platform', 'lw_gv_3p', 'lw_gv_3p_total',
       'lw_gv_platform_total', 'spn_flag', 'lw_gv_3p_ytd',
       'lw_gv_platform_ytd', 'lw_gv_3p_total', 'lw_gv_platform_total',
       'ebc_asin_coverage_asins_created', 'tax_unblocked_gms', 'ncns',
       'providerid_19', 'unblocked_asin', 'compliance_unblocked_gms',
       'service_19'],
      dtype='object')

In [103]:
## Ad Optimization scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'ad_revenue',
        'acos_p']

adz_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

adz_rank=adz_rank[adz_rank['service_name']=='Advertising Optimization']
adz_rank=adz_rank.fillna(0)

##prepare the score

adz_rank['score']=( ytc_percent_in_week_num_adz_wt*adz_rank['ytc_in_week_numerator']
                    + orders_completed_adz_wt*adz_rank['total_4w_orders']
                    + unique_seller_adz_wt*adz_rank['adoption']
                    + avg_feedback_adz_wt*adz_rank['avg_feedback']
                    + ad_revenue_adz_wt*adz_rank['ad_revenue']
                    + acos_adz_wt*adz_rank['acos_p']
                   )

adz_rank['rank']=adz_rank[['ad_revenue','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
adz_rank = adz_rank.sort_values(by="rank", ignore_index=True)
adz_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,ad_revenue,acos_p,score,rank
0,860468c6-9e94-4627-8e89-23e58e944fff,Advertising Optimization,0.0,340.0,0.0,4.33,56396657.46,617.916582,1.409929e+07,1.0
1,ac1398cb-72ca-443c-8ae6-8f764e1297f4,Advertising Optimization,0.0,1.0,0.0,0.00,25246498.68,205.900725,6.311645e+06,2.0
2,a021786e-3c21-48be-955a-549382f3a3ae,Advertising Optimization,0.0,8.0,0.0,4.50,17456135.90,294.438462,4.364066e+06,3.0
3,8efd01b7-d87b-4ba9-acf1-172256164c85,Advertising Optimization,0.0,0.0,0.0,4.00,9829457.47,34.611810,2.457368e+06,4.0
4,2a64ecda-082d-4361-b5c4-2cad8740c59d,Advertising Optimization,1.0,21.0,0.0,4.33,9146934.92,148.454509,2.286754e+06,5.0


In [104]:
## let's have the weights for account management in this block

ytc_percent_in_week_num_am_wt= 20/100
orders_completed_am_wt= 20/100
unique_seller_am_wt= 15/100
avg_feedback_am_wt= 15/100
am_gms_wt=30/100

In [105]:
## AM scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'boost_gms']

am_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

am_rank=am_rank[am_rank['service_name']=='Account Management']
am_rank=am_rank.fillna(0)

##prepare the score

am_rank['score']=( ytc_percent_in_week_num_am_wt*am_rank['ytc_in_week_numerator']
                    + orders_completed_am_wt*am_rank['total_4w_orders']
                    + unique_seller_am_wt*am_rank['adoption']
                    + avg_feedback_am_wt*am_rank['avg_feedback']
                    + am_gms_wt*am_rank['boost_gms']
                   )

am_rank['rank']=am_rank[['boost_gms','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
am_rank = am_rank.sort_values(by="rank", ignore_index=True)
am_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,boost_gms,score,rank
0,04407379-d775-4a69-9d63-bf5b5e01b282,Account Management,0.0,144.0,6.0,4.93,1.309096e+08,3.927291e+07,1.0
1,90fb720f-19d1-4bab-b1ab-c918355b7b4d,Account Management,0.0,17.0,0.0,5.00,9.482390e+07,2.844718e+07,2.0
2,860468c6-9e94-4627-8e89-23e58e944fff,Account Management,0.0,124.0,0.0,4.50,7.779096e+07,2.333731e+07,3.0
3,a021786e-3c21-48be-955a-549382f3a3ae,Account Management,0.0,11.0,1.0,4.81,6.801491e+07,2.040448e+07,4.0
4,b3f2f78a-5443-48ea-8b17-28b337100ccc,Account Management,0.0,4.0,0.0,3.00,5.441753e+07,1.632526e+07,5.0


In [106]:
## let's have the weights for EBC in this block

ytc_percent_in_week_num_ebc_wt= 15/100
orders_completed_ebc_wt= 20/100
unique_seller_ebc_wt= 15/100
avg_feedback_ebc_wt= 15/100
ebc_gv_wt=25/100
ebc_asin_coverage_wt=10/100

In [107]:
## EBC scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'ebc_gv_asins_created',
        'lw_gv_platform']

ebc_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

ebc_rank=ebc_rank[ebc_rank['service_name']=='Enhanced Brand Content']
ebc_rank=ebc_rank.fillna(0)

##prepare the score

ebc_rank['score']=( ytc_percent_in_week_num_ebc_wt*ebc_rank['ytc_in_week_numerator']
                    + orders_completed_ebc_wt*ebc_rank['total_4w_orders']
                    + unique_seller_ebc_wt*ebc_rank['adoption']
                    + avg_feedback_ebc_wt*ebc_rank['avg_feedback']
                    + ebc_gv_wt*ebc_rank['lw_gv_platform']
                    + ebc_asin_coverage_wt*ebc_rank['ebc_gv_asins_created']
                   )

ebc_rank['rank']=ebc_rank[['lw_gv_platform','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
ebc_rank = ebc_rank.sort_values(by="rank", ignore_index=True)
ebc_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,ebc_gv_asins_created,lw_gv_platform,score,rank
0,860468c6-9e94-4627-8e89-23e58e944fff,Enhanced Brand Content,0.0,0.0,0.0,5.0,55654.0,2354998.0,594315.65,1.0
1,a3edbea2-0b75-494c-80b9-d8fa529f2b31,Enhanced Brand Content,0.0,0.0,0.0,5.0,53807.0,1915960.0,484371.45,2.0
2,e624d8bb-038f-4e19-97bf-0f949769944b,Enhanced Brand Content,0.0,0.0,0.0,0.0,8483.0,895313.0,224676.55,3.0
3,f6ab326d-bcd5-4f98-bd54-3c058782f049,Enhanced Brand Content,0.0,0.0,0.0,0.0,3681.0,716220.0,179423.10,4.0
4,26ddd595-c3a2-413f-bec2-2de54264ede8,Enhanced Brand Content,0.0,0.0,0.0,5.0,4287.0,547318.0,137258.95,5.0


In [108]:
## let's have the weights for SNR in this block

ytc_percent_in_week_num_snr_wt= 20/100
#pip_percent_snr_wt= 10/100
orders_completed_snr_wt= 20/100
unique_seller_snr_wt= 15/100
avg_feedback_snr_wt= 15/100
snr_reinstatement_p_wt=30/100

In [109]:
## SNR scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'snr_reinst_p']

snr_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

snr_rank=snr_rank[snr_rank['service_name']=='Seller Reinstatement']
snr_rank=snr_rank.fillna(0)

##prepare the score

snr_rank['score']=( ytc_percent_in_week_num_snr_wt*snr_rank['ytc_in_week_numerator']
                    + orders_completed_snr_wt*snr_rank['total_4w_orders']
                    + unique_seller_snr_wt*snr_rank['adoption']
                    + avg_feedback_snr_wt*snr_rank['avg_feedback']
                    + snr_reinstatement_p_wt*snr_rank['snr_reinst_p']
                   )

snr_rank['rank']=snr_rank[['snr_reinst_p','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
snr_rank = snr_rank.sort_values(by="rank", ignore_index=True)
snr_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,snr_reinst_p,score,rank
0,5f7723a8-2107-419b-af0a-0daf23a9ccb1,Seller Reinstatement,0.0,5.0,5.0,5.0,1.0000,2.80000,1.0
1,8e00f8f9-88b5-4afd-b2e3-904b58abab44,Seller Reinstatement,0.0,5.0,5.0,0.0,1.0000,2.05000,2.0
2,cd11dd3a-1103-46b8-a25e-ea02580181f3,Seller Reinstatement,0.0,27.0,27.0,0.0,0.3333,9.54999,3.0
3,04407379-d775-4a69-9d63-bf5b5e01b282,Seller Reinstatement,0.0,103.0,86.0,5.0,0.1852,34.30556,4.0
4,90fb720f-19d1-4bab-b1ab-c918355b7b4d,Seller Reinstatement,0.0,513.0,513.0,0.0,0.0000,179.55000,5.0


In [110]:
## let's have the weights for DS in this block

ytc_percent_in_week_num_ds_wt= 20/100
#pip_percent_snr_wt= 10/100
orders_completed_ds_wt= 20/100
unique_seller_ds_wt= 15/100
avg_feedback_ds_wt= 15/100
ds_ncns_p_wt=30/100

In [111]:
## DS scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'ncns']

ds_rank = join_1_20[colsss]

## filter for the service and replace all the blanks with NAN values

ds_rank=ds_rank[ds_rank['service_name']=='Domestic Shipping']
ds_rank=ds_rank.fillna(0)

##prepare the score

ds_rank['score']=( ytc_percent_in_week_num_ds_wt*ds_rank['ytc_in_week_numerator']
                    + orders_completed_ds_wt*ds_rank['total_4w_orders']
                    + unique_seller_ds_wt*ds_rank['adoption']
                    + avg_feedback_ds_wt*ds_rank['avg_feedback']
                    + ds_ncns_p_wt*ds_rank['ncns']
                   )

ds_rank['rank']=ds_rank[['ncns','total_4w_orders','avg_feedback','adoption','ytc_in_week_numerator']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
ds_rank = ds_rank.sort_values(by="rank", ignore_index=True)
ds_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,ncns,score,rank
0,9aec9e35-135b-417e-af34-ce19772b8edd,Domestic Shipping,20.0,0.0,0.0,0.0,0,4.0,1.0
1,186b1aa4-24a8-454a-8aea-d4517546408e,Domestic Shipping,7.0,0.0,0.0,0.0,0,1.4,2.0
2,9c9912f9-76bc-4e16-9396-ae01ea4518bf,Domestic Shipping,6.0,0.0,0.0,0.0,0,1.2,3.0
3,6a9c04cb-4c34-4fa6-8d4a-b65561aeec1c,Domestic Shipping,4.0,0.0,0.0,0.0,0,0.8,4.0
4,57df9332-9ed3-4266-8408-097fb7e21941,Domestic Shipping,4.0,0.0,0.0,0.0,0,0.8,4.0


In [112]:
## concatenate all the services and rank in 1 view

final_rank_sss= pd.concat([
    imag_rank[['provider_id','service_name','rank']],
    accounting_rank[['provider_id','service_name','rank']],
    training_rank[['provider_id','service_name','rank']],
    i_s_rank[['provider_id','service_name','rank']],
    trans_rank[['provider_id','service_name','rank']],
    i_r_rank[['provider_id','service_name','rank']],
    fba_rank[['provider_id','service_name','rank']],
    stor_rank[['provider_id','service_name','rank']],
    ipa_rank[['provider_id','service_name','rank']],
    exec_rank[['provider_id','service_name','rank']],
    cat_rank[['provider_id','service_name','rank']],
    tax_rank[['provider_id','service_name','rank']],
    comp_rank[['provider_id','service_name','rank']],
    adz_rank[['provider_id','service_name','rank']],
    am_rank[['provider_id','service_name','rank']],
    ebc_rank[['provider_id','service_name','rank']],
    snr_rank[['provider_id','service_name','rank']],
    ds_rank[['provider_id','service_name','rank']]

    ],ignore_index=True)

final_rank_sss.head()

,provider_id,service_name,rank
0,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Imaging,1.0
1,a5c32bce-5fe4-4f64-8764-7987f12eab79,Imaging,2.0
2,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Imaging,3.0
3,de030dd5-7b08-4987-8f4d-554aaf98f8f7,Imaging,4.0
4,348d6610-bf8c-40e0-985e-4c913b7d1f78,Imaging,5.0


In [113]:
final_rank_sss.to_clipboard()

In [116]:
## have to check out on caq empty and the proviider ids for cat
## NCNS is giving empty results have to check that
## we can use the new ad rev query and take a provider level cut and same we can do for boost gms
## have to use dense ranking instead of normal ranking

In [115]:
## consolidate all the data frames into 1 single report
path= r'C:\Users\arvinmvn\Desktop\Ranking\ranking_in_output_python.xlsx'

writer = pd.ExcelWriter(path,engine='xlsxwriter')
# Write each dataframe to a different worksheet.
final_rank_sss.to_excel(writer, sheet_name='final_rank',header = True,index=False)
join_1_20.to_excel(writer, sheet_name='metrics_raw',header = True,index=False)
imag_rank.to_excel(writer, sheet_name='imaging',header = True,index=False)
accounting_rank.to_excel(writer, sheet_name='accounting',header = True,index=False)
training_rank.to_excel(writer, sheet_name='training',header = True,index=False)
i_s_rank.to_excel(writer, sheet_name='international_shipping',header = True,index=False)
trans_rank.to_excel(writer, sheet_name='translation',header = True,index=False)
i_r_rank.to_excel(writer, sheet_name='international retuns',header = True,index=False)
fba_rank.to_excel(writer, sheet_name='fba preparation',header = True,index=False)
stor_rank.to_excel(writer, sheet_name='storage',header = True,index=False)
ipa_rank.to_excel(writer, sheet_name='ip accelarator',header = True,index=False)
exec_rank.to_excel(writer, sheet_name='excess inventory',header = True,index=False)
cat_rank.to_excel(writer, sheet_name='cataloguing',header = True,index=False)
tax_rank.to_excel(writer, sheet_name='taxes',header = True,index=False)
comp_rank.to_excel(writer, sheet_name='compliance',header = True,index=False)
adz_rank.to_excel(writer, sheet_name='ad optimization',header = True,index=False)
am_rank.to_excel(writer, sheet_name='account management',header = True,index=False)
ebc_rank.to_excel(writer, sheet_name='ebc',header = True,index=False)
snr_rank.to_excel(writer, sheet_name='snr',header = True,index=False)
ds_rank.to_excel(writer, sheet_name='ds',header = True,index=False)
writer.save()

C:\Users\arvinmvn\AppData\Local\Temp\ipykernel_11976\402810154.py:26: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
